https://zerojsh00.github.io/posts/BERTopic/
https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6
https://arxiv.org/pdf/2203.05794.pdf

In [11]:
! which python

/Users/cslab/anaconda3/bin/python


In [100]:
pip install bertopic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 154 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 922 kB/s eta 0:00:01     |███████████████████████████▍    | 13.4 MB 438 kB/s eta 0:00:06
     |████████████████████████████████| 163 kB 1.5 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [61]:
pip uninstall umap


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: umap 0.1.1
Uninstalling umap-0.1.1:
  Would remove:
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap-0.1.1.dist-info/*
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/*
  Would not remove (might be manually added):
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/aligned_umap.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/distances.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/layouts.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/parametric_umap.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/plot.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/sparse.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/spectral.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/umap_.py
    /Users/cslab/Library/Python/3.9/lib/python/site-packages/umap/utils.py
    /Users/cslab/Lib

In [91]:
pip install hdbscan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.2 MB 806 kB/s eta 0:00:01
  distutils: /private/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/pip-build-env-46d3sf_0/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/pip-build-env-46d3sf_0/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/pip-build-env-46d3sf_0/normal'
  distutils: /private/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/pip-build-env-46d3sf_0/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/pip-build-env-46d3sf_0/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = 

In [ ]:
pip install umap-learn

In [98]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
import umap.umap_ as umap
import hdbscan
from ctfidf import CTFIDFVectorizer

ModuleNotFoundError: No module named 'ctfidf'

In [13]:
# 파일 불러오기
with open('../../data/bert_df.pkl', 'rb') as f:
    df = pickle.load(f)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3487 entries, 0 to 3486
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   q_id                3487 non-null   int64         
 1   q_posttypeid        3487 non-null   object        
 2   q_acceptedanswerid  1488 non-null   float64       
 3   q_parentid          0 non-null      object        
 4   q_creationdate      3487 non-null   datetime64[ns]
 5   q_score             3487 non-null   int64         
 6   q_viewcount         3487 non-null   int64         
 7   q_owneruserid       3487 non-null   int64         
 8   q_title             3487 non-null   object        
 9   q_tags              3487 non-null   object        
 10  q_answercount       3487 non-null   int64         
 11  q_commentcount      3487 non-null   int64         
 12  q_reputation        3487 non-null   int64         
 13  q_text              3487 non-null   object      

In [15]:
# extract the  columns for text analysis
## q_body : question body
## a_body : answer body
df_qna = df[['q_id','q_text', 'a_text']]

In [16]:
df_qna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3487 entries, 0 to 3486
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   q_id    3487 non-null   int64 
 1   q_text  3487 non-null   object
 2   a_text  3487 non-null   object
dtypes: int64(1), object(2)
memory usage: 81.9+ KB


In [17]:
def cleanhtml(raw_html):
  # 1.Source code in python language is hard to understand, so replace all the <code> tag first
  cleantext_1 = re.findall(r'(?<=\<code>)(.*?)(?=<\/code>)', raw_html.replace('\n', '_**_'))
  cleantext_1 = [x.replace('_**_', '\n') for x in cleantext_1]
  # 2. replace html tags
  # <p>
  tag_re = re.compile('<.*?>')
  cleantext_2 = [re.sub(tag_re, '', x) for x in cleantext_1]
  return cleantext_2

In [18]:
# apply the function, cleanhtml to the question and body text
df_qna.loc[:, 'q_prep_text'] = df_qna['q_text'].apply(cleanhtml)
df_qna.loc[:, 'a_prep_text'] = df_qna['a_text'].apply(cleanhtml)

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_15110/1671787946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qna.loc[:, 'q_prep_text'] = df_qna['q_text'].apply(cleanhtml)
/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_15110/1671787946.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qna.loc[:, 'a_prep_text'] = df_qna['a_text'].apply(cleanhtml)


In [19]:
df_qna.loc[:, [ 'q_id','q_prep_text', 'a_prep_text']] 

,q_id,q_prep_text,a_prep_text
0,77589765,"[M, N, n, M, M, N, M * N[None, None, :, None, ...","[In [29]: a,b,n,e,f = 2,3,4,5,6\nIn [30]: M=np..."
1,77591165,[import numpy as np\nimport pandas as pd\n\nim...,"[f = [client.submit(merge_noindex, d1, d2) \n ..."
2,77550047,[import pyautogui\npyautogui.displayMousePosit...,[]
3,77591174,"[list1=[0, 1, None, None, None, None, None, No...",[while len(fibonacci_sequence) &lt; n:\n fi...
4,77591245,"[__del__, __del__, cache = []\n\nclass Temp:\n...","[__del__, __del__, __del__, cache, atexit]"
...,...,...,...
3482,77591785,[indicadores_clean_pv = pd.melt(indicadores_cl...,"[.stack(), .unstack(), out = (\n df.set_ind..."
3483,77591847,"[4\n6\n7\n0\n, .XXX\n.X.X\n...X\nXXXX\n\n.XXXX...","[spiral_rec_southwest(M, xmin, xmax, ymin, yma..."
3484,77592073,[],[]
3485,77592121,[import random \n \n \nclass Pl...,"[select_hero(), import random \n ..."


In [20]:
df_q_src = df_qna[['q_id', 'q_prep_text']].apply(pd.Series.explode)
df_a_src = df_qna[['q_id', 'a_prep_text']].apply(pd.Series.explode)

In [29]:
df_q_src = df_q_src.reset_index(drop=True)

In [47]:
src_list = df_q_src.loc[df_q_src['q_id'] ==77592121, 'q_prep_text'].str.replace('\n', '').values[0]
src_list

'import random                  class Player:    def __init__(self, name, weapon, strength, health, attack, hero, creature):        self.name = name        self.weapon = weapon        self.strength = 100        self.health = 100        self.attack = 0        self.hero = hero        self.creature = creature            def entername_player(self):        self.name = input(&quot;Enter your name: &quot;)        print(&quot;Welcome &quot; + self.name + &quot; !&quot;)        return self.name                  def main():        hero = entername_player()                                    if __name__ == &quot;__main__&quot;:        main()    '

In [22]:
from transformers import AutoTokenizer, AutoModel
import torch

/Users/cslab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [83]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
nl_tokens=tokenizer.tokenize("return maximum value")

code_tokens=tokenizer.tokenize(src_list)

tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.eos_token]

tokens_ids=tokenizer.convert_tokens_to_ids(tokens)

context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]


In [87]:
context_embeddings = context_embeddings.reshape(-1,768)

In [88]:
context_embeddings.shape

torch.Size([323, 768])

In [89]:

umap_embeddings = umap.UMAP(n_neighbors=15,
                            n_components=5,
                            metric='cosine').fit_transform(context_embeddings.detach().numpy())




In [93]:
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',
                          cluster_selection_method='eom').fit(umap_embeddings)

In [94]:
cluster

HDBSCAN(min_cluster_size=15)

In [24]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

CODE = "if (x is not None) <mask> (x>1)"
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
print(outputs)

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/codebert-base-mlm were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

[{'score': 0.9424599409103394, 'token': 8, 'token_str': ' and', 'sequence': 'if (x is not None) and (x>1)'}, {'score': 0.029372703284025192, 'token': 50, 'token_str': ' or', 'sequence': 'if (x is not None) or (x>1)'}, {'score': 0.0065606399439275265, 'token': 114, 'token_str': ' if', 'sequence': 'if (x is not None) if (x>1)'}, {'score': 0.004798914305865765, 'token': 48200, 'token_str': ' &&', 'sequence': 'if (x is not None) && (x>1)'}, {'score': 0.0031976045574992895, 'token': 172, 'token_str': ' then', 'sequence': 'if (x is not None) then (x>1)'}]
